<font color='darkred'> Unless otherwise noted, **this notebook will not be reviewed or autograded.**</font> You are welcome to use it for scratchwork, but **only the files listed in the exercises will be checked.**

---

# Exercises

For these exercises, you'll be creating a [Python class](https://www.hackerearth.com/practice/python/object-oriented-programming/classes-and-objects-i/tutorial/) in the *apputil.py* file.

A few tips:

- Some of these exercises will require some "JSON digging".
- <font color='lightblue'>Your functions should still adhere to [best practices](https://codesignal.com/learn/courses/clean-code-basics-with-python/lessons/clean-function-design-in-python), so **your class should probably contain more functions than only the ones dictated below**.</font>


## Exercise 1

Create a Python class named `Genius` such that the following code initializes the object, and "saves" the access token as an attribute of the object. You'll need to use this attribute for Exercises 2 and 3.

```python
from apputil import Genius

genius = Genius(access_token="access_token")
```

In [ ]:
# apputil.py

class Genius:
    def __init__(self, access_token):
        self.access_token = access_token

## Exercise 2

Reference the `json_data` in the lab. Notice, when we search for an artist name (e.g., "Missy Elliot") in Genius, the result is a list of *songs* attributed to that artist. Suppose we want to capture information about the artist themselves.

Create a method for our `Genius` class called `.get_artist(search_term)` which does the following:

1. Extract the (most likely, "Primary") Artist ID from the first "hit" of the `search_term`.
2. Use the [API path](https://docs.genius.com/#artists-h2) for this Artist ID to pull information about the artist.
3. **Return** the dictionary containing the resulting JSON object.

For example, the following code should return a dictionary of artist information:

```python
genius.get_artist("Radiohead")
```

In [ ]:
# apputil.py
import requests

class Genius:
    def __init__(self, access_token):
        self.access_token = access_token
        self.base_url = "https://api.genius.com"

    def get_artist(self, search_term):
        headers = {
            "Authorization": f"Bearer {self.access_token}"
        }

        # Step 1: Search for the artist
        search_url = f"{self.base_url}/search"
        params = {"q": search_term}
        response = requests.get(search_url, headers=headers, params=params)
        data = response.json()

        # Step 2: Extract the primary artist ID from the first hit
        first_hit = data["response"]["hits"][0]
        artist_id = first_hit["result"]["primary_artist"]["id"]

        # Step 3: Use the artist ID to get artist info
        artist_url = f"{self.base_url}/artists/{artist_id}"
        artist_response = requests.get(artist_url, headers=headers)
        artist_data = artist_response.json()

        # Step 4: Return the artist data dictionary
        return artist_data

## Exercise 3

Use the result from Exercise 2 to create another method for our `Genius` class called `.get_artists(search_terms)` (plural) which takes in a *list* of search terms, and returns a DataFrame containing a row for each search term, and the following columns:

- `search_term`: the raw search term from `search_terms`
- `artist_name`: the (most likely) artist name for the search term
- `artist_id`: the Genius Artist ID for that artist, based on the API call
- `followers_count`: the number of followers for that artist (if available)

For example, the following should return a DataFrame with 4 rows:

```python
genius.get_artists(['Rihanna', 'Tycho', 'Seal', 'U2'])
```

In [ ]:
# apputil.py
import requests
import pandas as pd

class Genius:
    def __init__(self, access_token):
        self.access_token = access_token
        self.base_url = "https://api.genius.com"

    def get_artist(self, search_term):
        headers = {
            "Authorization": f"Bearer {self.access_token}"
        }

        search_url = f"{self.base_url}/search"
        params = {"q": search_term}
        response = requests.get(search_url, headers=headers, params=params)
        data = response.json()

        first_hit = data["response"]["hits"][0]
        artist_id = first_hit["result"]["primary_artist"]["id"]

        artist_url = f"{self.base_url}/artists/{artist_id}"
        artist_response = requests.get(artist_url, headers=headers)
        artist_data = artist_response.json()

        return artist_data

    def get_artists(self, search_terms):
        results = []

        for term in search_terms:
            try:
                artist_data = self.get_artist(term)
                artist_info = artist_data["response"]["artist"]

                results.append({
                    "search_term": term,
                    "artist_name": artist_info.get("name"),
                    "artist_id": artist_info.get("id"),
                    "followers_count": artist_info.get("followers_count")
                })
            except Exception as e:
                results.append({
                    "search_term": term,
                    "artist_name": None,
                    "artist_id": None,
                    "followers_count": None
                })

        return pd.DataFrame(results)

## Bonus Exercise (optional)

1. Gather a list of 100+ various musical artists, and save this list in a TXT file.
2. Write a Python *script* that saves the result of `.get_artists` for these artists in a CSV file.
3. If you have time, adjust this script to use multiprocessing.